In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
audio_dt = pd.read_pickle('data/imported_audio.pkl')
audio_dt = audio_dt[audio_dt.esc10]

In [9]:
audio_dt.head()

,filename,audio,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,<0/1-100032-A-0.wav>,1,0,dog,True,100032,A
14,1-110389-A-0.wav,<0/1-110389-A-0.wav>,1,0,dog,True,110389,A
24,1-116765-A-41.wav,<41/1-116765-A-41.wav>,1,41,chainsaw,True,116765,A
54,1-17150-A-12.wav,<12/1-17150-A-12.wav>,1,12,crackling_fire,True,17150,A
55,1-172649-A-40.wav,<40/1-172649-A-40.wav>,1,40,helicopter,True,172649,A


## Plot the followings:
- Historgram of each coefficient for each class
- Random plot of N classes

In [10]:
dict_category = {}

In [11]:
row = audio_dt.iloc[0]

In [15]:
row

filename        1-100032-A-0.wav
audio       <0/1-100032-A-0.wav>
fold                           1
target                         0
category                     dog
esc10                       True
src_file                  100032
take                           A
Name: 0, dtype: object

In [13]:
dict_category[row['category']] = []

KeyError: 'dog'

In [6]:
audio_dt['audio'][0].mfcc[0]

array([-300.99148845,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ])